In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import random
import string
import pyperclip
import re
from typing import Set

class PasswordGenerator:
    def __init__(self, root):
        self.root = root
        self.root.title("Advanced Password Generator")
        self.root.geometry("600x700")
        
       
        self.style = ttk.Style()
        self.style.configure('TButton', padding=5)
        self.style.configure('TCheckbutton', padding=2)
        
        
        self.password_var = tk.StringVar()
        self.length_var = tk.StringVar(value="12")
        self.uppercase_var = tk.BooleanVar(value=True)
        self.lowercase_var = tk.BooleanVar(value=True)
        self.numbers_var = tk.BooleanVar(value=True)
        self.symbols_var = tk.BooleanVar(value=True)
        self.exclude_similar_var = tk.BooleanVar(value=True)
        self.exclude_chars_var = tk.StringVar()
        
        self.create_widgets()
        
    def create_widgets(self):
       
        main_frame = ttk.Frame(self.root, padding="20")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
       
        password_frame = ttk.LabelFrame(main_frame, text="Generated Password", padding="10")
        password_frame.pack(fill=tk.X, pady=10)
        
        password_entry = ttk.Entry(
            password_frame,
            textvariable=self.password_var,
            font=('Courier', 14),
            justify='center',
            state='readonly'
        )
        password_entry.pack(fill=tk.X, pady=5)
        
        
        copy_btn = ttk.Button(
            password_frame,
            text="Copy to Clipboard",
            command=self.copy_to_clipboard
        )
        copy_btn.pack(pady=5)
        
        
        settings_frame = ttk.LabelFrame(main_frame, text="Password Settings", padding="10")
        settings_frame.pack(fill=tk.X, pady=10)
        
        length_frame = ttk.Frame(settings_frame)
        length_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(length_frame, text="Password Length:").pack(side=tk.LEFT)
        length_entry = ttk.Entry(
            length_frame,
            textvariable=self.length_var,
            width=5
        )
        length_entry.pack(side=tk.LEFT, padx=5)
        
        
        ttk.Checkbutton(
            settings_frame,
            text="Include Uppercase Letters (A-Z)",
            variable=self.uppercase_var
        ).pack(fill=tk.X, pady=2)
        
        ttk.Checkbutton(
            settings_frame,
            text="Include Lowercase Letters (a-z)",
            variable=self.lowercase_var
        ).pack(fill=tk.X, pady=2)
        
        ttk.Checkbutton(
            settings_frame,
            text="Include Numbers (0-9)",
            variable=self.numbers_var
        ).pack(fill=tk.X, pady=2)
        
        ttk.Checkbutton(
            settings_frame,
            text="Include Symbols (!@#$%^&*)",
            variable=self.symbols_var
        ).pack(fill=tk.X, pady=2)
        
       
        advanced_frame = ttk.LabelFrame(main_frame, text="Advanced Options", padding="10")
        advanced_frame.pack(fill=tk.X, pady=10)
        
        ttk.Checkbutton(
            advanced_frame,
            text="Exclude Similar Characters (l, 1, I, 0, O)",
            variable=self.exclude_similar_var
        ).pack(fill=tk.X, pady=2)
        
        ttk.Label(advanced_frame, text="Exclude Specific Characters:").pack(fill=tk.X, pady=2)
        ttk.Entry(
            advanced_frame,
            textvariable=self.exclude_chars_var
        ).pack(fill=tk.X, pady=2)
        
      
        generate_btn = ttk.Button(
            main_frame,
            text="Generate Password",
            command=self.generate_password
        )
        generate_btn.pack(pady=10)
        
        
        self.strength_var = tk.StringVar(value="Password Strength: N/A")
        strength_label = ttk.Label(
            main_frame,
            textvariable=self.strength_var,
            font=('Default', 10, 'bold')
        )
        strength_label.pack(pady=5)
        
    def get_character_set(self) -> Set[str]:
        chars = set()
        
        if self.uppercase_var.get():
            chars.update(string.ascii_uppercase)
        if self.lowercase_var.get():
            chars.update(string.ascii_lowercase)
        if self.numbers_var.get():
            chars.update(string.digits)
        if self.symbols_var.get():
            chars.update(string.punctuation)
            
        if self.exclude_similar_var.get():
            chars -= set('l1IO0')
            
        exclude_chars = set(self.exclude_chars_var.get())
        chars -= exclude_chars
        
        return chars
        
    def generate_password(self):
        try:
            length = int(self.length_var.get())
            if length < 4:
                raise ValueError("Password length must be at least 4 characters")
                
            chars = self.get_character_set()
            if not chars:
                raise ValueError("Please select at least one character type")
                
            
            password = []
            if self.uppercase_var.get():
                password.append(random.choice(string.ascii_uppercase))
            if self.lowercase_var.get():
                password.append(random.choice(string.ascii_lowercase))
            if self.numbers_var.get():
                password.append(random.choice(string.digits))
            if self.symbols_var.get():
                password.append(random.choice(string.punctuation))
                
            
            remaining_length = length - len(password)
            password.extend(random.choice(list(chars)) for _ in range(remaining_length))
            
          
            random.shuffle(password)
            password = ''.join(password)
            
            self.password_var.set(password)
            self.update_strength_meter(password)
            
        except ValueError as e:
            messagebox.showerror("Error", str(e))
            
    def update_strength_meter(self, password: str):
        score = 0
        feedback = []
        
      
        if len(password) >= 12:
            score += 2
            feedback.append("Good length")
        elif len(password) >= 8:
            score += 1
            feedback.append("Acceptable length")
        else:
            feedback.append("Too short")
            
       
        if re.search(r'[A-Z]', password):
            score += 1
        if re.search(r'[a-z]', password):
            score += 1
        if re.search(r'[0-9]', password):
            score += 1
        if re.search(r'[^A-Za-z0-9]', password):
            score += 1
            
        
        if score >= 5:
            strength = "Strong"
        elif score >= 3:
            strength = "Medium"
        else:
            strength = "Weak"
            
        self.strength_var.set(f"Password Strength: {strength}")
        
    def copy_to_clipboard(self):
        password = self.password_var.get()
        if password:
            pyperclip.copy(password)
            messagebox.showinfo("Success", "Password copied to clipboard!")
        else:
            messagebox.showwarning("Warning", "No password to copy!")

if __name__ == "__main__":
    root = tk.Tk()
    app = PasswordGenerator(root)
    root.mainloop()